### 🚀 Agentic AI:  Smart Legal Contract Assistant

In [1]:
# LangChain Agents – Agentic AI for reasoning & planning.
# PyMuPDF / pdfplumber – Extracts text from PDFs.
# OpenAI GPT-4 / Llama-2 – For smart reasoning.
# FAISS / ChromaDB – Vector storage for retrieval.
# Python Functions – AI decides which function to call.

In [8]:
!pip install langchain langchain-community llama-index openai pypdf pdfplumber chromadb faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:00


In [30]:
import pdfplumber
from langchain.tools import tool

In [34]:
def extract_text(pdf_path: str) -> str:
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text[:10000]

In [82]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.tools import Tool, StructuredTool


In [63]:
def vec_database(text: str):
  textsplitter = RecursiveCharacterTextSplitter(separators=["\n"], chunk_size=1000, chunk_overlap=100)
  texts = textsplitter.split_text(text)
  documents = [Document(page_content=t) for t in texts]

  vector_store = FAISS.from_documents(documents, OpenAIEmbeddings(api_key="********************************"))
  return vector_store


In [73]:
# Query Vector DB
def query_vector_db(vector_db, query):
    retriever = vector_db.as_retriever()
    llm = ChatOpenAI(model_name="gpt-4", temperature=0, api_key="*********************************")
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
    return qa_chain.invoke({"query": query})  # Updated from .run() to .invoke()

In [64]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [65]:
# Load LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0, api_key="******************************")

# Add Memory (for tracking progress)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [94]:
from typing import Dict, Any

In [88]:
from langchain.tools import Tool

In [96]:
# Add Tools
# Define tools using the Tool class from langchain
tools = [
    Tool(
        name="extract_text",
        func=extract_text,
        description="Extracts text from a PDF file. Input should be the path to the PDF file.",
        args_schema=None  # or a pydantic schema if you want to define argument types
    ),
    Tool(
        name="vec_database",
        func=vec_database,
        description="Creates a vector database from a given text. Input should be the text to be indexed.",
        args_schema=None  # or a pydantic schema
    ),
    StructuredTool(
        name = "query_vector_db",
        # func = query_vector_db,
        func = lambda x: query_vector_db(x["vector_db"], x["query"]), # update the function call
        description = "Queries a vector database. Input should be the vector database and the query.",
        args_schema = None
    )
]


In [97]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

In [86]:
# ✅ USER PROVIDES DOCUMENT & QUERY
pdf_path = "/content/MUM-0225-CL-0005187-S01.pdf"
user_query = "What is Name of Insurance Company?"

# ✅ Ensure Document is Processed Before Querying
pdf_text = extract_text(pdf_path)  # Step 1: Extract
vector_db = vec_database(pdf_text)  # Step 2: Store in DB

# ✅ Give the Agent Context Before Querying
agent.memory.chat_memory.add_user_message("Here is the contract text: " + pdf_text[:2000])  # Limiting for tokens

# ✅ Now Query with the Context
response = query_vector_db(vector_db, user_query)

print("🔹 AI Response:", response)

🔹 AI Response: {'query': 'What is Name of Insurance Company?', 'result': 'The name of the insurance company is not provided in the given context.'}


In [104]:
# ✅ USER PROVIDES DOCUMENT & QUERY
pdf_path = "/content/MUM-0225-CL-0005187-S01.pdf"
user_query = "What is Patient name?"

# ✅ Agent Runs the Full Pipeline
response = agent.invoke({"input": f"Analyze the document {pdf_path} and answer: {user_query}"})




> Entering new AgentExecutor chain...
Thought: The first step is to extract the text from the PDF document. I will use the extract_text tool for this.

Action:
```
{
  "action": "extract_text",
  "action_input": "/content/MUM-0225-CL-0005187-S01.pdf"
}
```
Observation: To,
EXELA TECHNOLOGIES INDIA PVT. LTD Date: 19/02/2025
BACS12364 Shortfall Letter No: MUM-0225-CL-0005187-S01
Address U/W Claim Number:
T 341 INTERNATIONAL INFOTECH PARK VASHI NAVI Agent Code:
Dev off Code:
MUMBAI,MAHARASHTRA
Pin code: 400705
Phone no: 8197320658
Sub:Query pertaining to claim MUM-0225-CL-0005187
Proposer/Insured Name EXELA TECHNOLOGIES INDIA PVT. LTD/TUTIKA PRAKASH KUMAR
Policy number 50780300202400
Employee Code BACS12364 ID Card number MUM-MB-B0560-007-0000706-D
Patient name TUTIKA BENNAYYA LINGAM Hospital name APOLLO HOSPITAL
Date of Admission 23-Jan-2025 Date of Discharge 26-Jan-2025
Claim Amount 263500 Claim number MUM-0225-CL-0005187
Dear Sir/Madam,
With reference to your claim paper submitted wi

In [126]:
agent.memory.chat_memory.messages[11]

AIMessage(content="The patient's name is TUTIKA BENNAYYA LINGAM.", additional_kwargs={}, response_metadata={})